# VisList
#### Calculates visibility of a target from a given location over a set time window. Also considers the length of time target is visible for
#### Requries two files, one of locations, and one of targets, see example files supplied for formats, 'locations' and 'testfile'
#### Set target filename below, must be .in file, returns .out file of same name

In [1]:
target_file = 'testfile'

In [2]:
import numpy as np
from PyAstronomy import pyasl
import matplotlib.pyplot as plt
import datetime as dt
from astropy.coordinates import SkyCoord
import json

#### Create target object

In [3]:
#define target object
class Target:
    def __init__(self, target_name, obs_date, coordinates, obs_window, obs_duration):
        """Creates a target object from specified parameters
        :param target_name: target identifier
        :param obs_date: start of observation window
        :param coordinates: RA and Dec of target in SkyCoord object
        :param obs_window: length of observing window in days
        :param obs_duration: required amount of visible time for an observation in minutes
        """
        self.name = target_name
        self.date = obs_date
        self.coord = coordinates
        self.window = obs_window #number of days from date
        self.duration = int(obs_duration) #time visible in a night
    

#### Create location object

In [4]:
#define location object
class Location:
    def __init__(self, loc_name, latitude, longitude, loc_altitude, minimum_targ_alt, timezone):
        """Creates a location object from the specified parameters
        :param loc_name: observatory name
        :param latitude: observatory latitude
        :param longitude: observatory longitude
        :param loc_altitude: observatory altitude in m
        :param minimum_targ_alt: minimum altitude above horizon for observation in decimal deg
        :param timezone: UTC offset in hours X.X
        """
        self.loc = loc_name
        self.lati = float(latitude)
        self.long = float(longitude)
        self.alti = float(loc_altitude)
        self.targ_alt = minimum_targ_alt
        self.zone = float(timezone)

#### Load in location list from location file

In [5]:
locations=[] #list of observatories
keys=[] #observatory keys
locfilename = 'locations.dat' #location file
file = open(locfilename,'r')

#read file one line at a time, extract data from each
for line in file:
    if line[0] != '!': #check for comments
        tokens = []
        split = line.split()
        
        #extract information from line
        for token in split:
            tokens.append(token) 
        loc = tokens[0]
        lat = tokens[1]
        lon = tokens[2]
        alt = tokens[3]
        minimum = float(tokens[4])
        tzone = tokens[5]
        
        if tokens[0] not in keys: #check if location is already in list, only add if new
            keys.append(tokens[0]) 
            newloc = Location(loc, lat, lon, alt, minimum, tzone) #initialise location object
            locations.append(newloc) #add to list

#### Load in list of targets from file

In [6]:
file = open(target_file+'.in','r') #open input file

targets = [] #initialise list

for line in file: #read all lines in file
    if line[0] != '!':
        tokens = [] 
        split = line.split()
        
        #extract information from line
        for token in split:
            tokens.append(token)
        target = tokens[0]
        date = tokens[1]
        ra = tokens[2]
        dec = tokens[3]
        coord = SkyCoord(ra, dec) #convert RA and Dec to SkyCoord
        win = tokens[4]
        dur = tokens[5]
        
        newtarget = Target(target, date, coord, win, dur) #create new object
        targets.append(newtarget) #add to list

#### For each target, compute visibility from each location on each day, return result as json file

In [7]:
datewidth = 1.0/24/20 #define spacing of data points

output = open(target_file+'.out','w') #open output file

for i in range(0,len(targets)): #loop across targets
    ymd = [int(j) for j in targets[i].date.split('-')] #extract date
    
    #loop across locations
    for k in range(0, len(locations)):
        date = dt.datetime(*ymd) - dt.timedelta(hours = locations[k].zone) #adjust date for timezone
        j = 0 #initialise counter and loop across window
        while j < int(targets[i].window):
            j += 1 #increment
            visibility = False #initialise boolean
            
            jd = pyasl.jdcnv(date) #find julian date
            jd_start = jd-0.5 #start of time range
            jd_end = jd+0.5 #end of time range
            jds = np.arange(jd_start,jd_end, datewidth) #array of points across timerange
            jdsub = jds-np.floor(jds[0])

            #find sun position
            sunpos = pyasl.sunpos(jd)
            sun_ra, sun_dec = sunpos[1], sunpos[2]
            
            #find target and sun altazimuthal coordinates across timerange
            targ_altaz = pyasl.eq2hor(jds, np.ones(jds.size)*targets[i].coord.ra.deg, np.ones(jds.size)*targets[i].coord.dec.deg, lat=locations[k].lati, lon=locations[k].long, alt=locations[k].alti)
            sun_altaz = pyasl.eq2hor(jds, np.ones(jds.size)*sun_ra, np.ones(jds.size)*sun_dec, lat=locations[k].lati, lon=locations[k].long, alt=locations[k].alti)

            #find visible time points
            visible = np.where(np.logical_and(sun_altaz[0] < 0.0, targ_altaz[0] > locations[k].targ_alt))[0]

            #convert number of time points to minutes
            vis_dur_min = len(visible)*datewidth*24*60

            #check if target is visible for desired duration
            if vis_dur_min > targets[i].duration:
                    visibility = True #set boolean

            #create output line
            new_data = {targets[i].name+' '+locations[k].loc+' '+str(date.date()) : visibility}
            json_data = json.dumps(new_data, ensure_ascii=False) #convert to json
            print(new_data)
            output.write(json_data+'\n') #write to file
            
            date = date + dt.timedelta(days=1) #increment date
        
output.close() #close file

{'m81 UCLO 2018-09-11': True}
{'m81 UCLO 2018-09-12': True}
{'m81 UCLO 2018-09-13': True}
{'m81 UCLO 2018-09-14': True}
{'m81 UCLO 2018-09-15': True}
{'m81 AAO 2018-09-10': False}
{'m81 AAO 2018-09-11': False}
{'m81 AAO 2018-09-12': False}
{'m81 AAO 2018-09-13': False}
{'m81 AAO 2018-09-14': False}


In [8]:
### CODE FOR EXTRACTING TIMES, NEEDS FIXING:
### Does not currently agree with results from multipl

#if not visible.size:
    #continue
            
#else:   
    #extract start of visibility
    #vis_start = jdsub[visible][0]
    #vs_hour = str(int(str(vis_start*24)[0])+12) #convert from JD to decimal, take first digit as hour
    #vs_min = str(int(np.round(vis_start*24%1*60,2))) #convert from JD to decimal, take decimal, convert to minutes
    #vs_utc_str = vs_hour+':'+vs_min
    #vs_utc = dt.datetime.strptime(vs_utc_str, '%H:%M')

    #extract end of visibility
    #vis_end = jdsub[visible][len(jdsub[visible])-1]
    #ve_hour = str((vis_end-0.5)*24)[0] #convert from JD to decimal, take first digit as hour
    #ve_min = str(int(np.round(vis_end*24%1*60,2))) #convert from JD to decimal, take decimal, convert to minutes
    #ve_utc_str = ve_hour+':'+ve_min
    #ve_utc = dt.datetime.strptime(ve_utc_str, '%H:%M')

    #vis_dur = ve_utc - vs_utc
    #vis_dur_min = int(vis_dur.seconds / 60)